<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=da93eb4d614425838f1b1499a26f18a6f73b8adc0ed66b37dac2c0c6f70c723d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 13:06:37
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: -1.57 L (-1.13%)
Current PnL: -22.63 L (-15.22%)
CY Booked + Current PnL: -11.19 L (-7.52%)
-------------------
Total profit:  87.45 K
Total loss:  -23.50 L
-------------------
Total Booked + Current PnL: 15.79 L (12.98%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.35%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.49 L (64.61%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,92.96,35.0,M-SC,2.74,83100.0,-17677.0,17767.0,-2.89,-17.54,21.38,0.09,245.0,-0.99,0.61,9.17,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.63,47.0,H-MC,3.82,189170.0,24500.0,18822.0,-1.85,14.88,9.95,26.31,89.0,1.30,1.38,24.05,X40N,NTT,AC
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.91,106954.0,6450.0,22653.0,-1.85,6.42,21.18,28.96,79.0,0.28,0.78,42.56,MH,ATH,METALS
42,ITC,452.00,-38.77,46.0,H-LC,2.14,198143.0,-1995.0,22886.0,1.27,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.04,46.0,H-LC,4.08,215498.0,5132.0,23640.0,0.66,2.44,10.97,13.68,37.0,0.22,1.57,19.37,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.0,-22.16,44.0,M-LC,2.36,246023.0,-30500.0,140578.0,2.02,-11.03,57.14,39.81,54.0,-0.22,1.80,17.91,XY24,NTT,AUTO
42,ITC,452.0,-38.77,46.0,H-LC,2.14,198143.0,-1995.0,22886.0,1.27,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG
45,KANSAINER,340.0,-67.79,53.0,H-SC,1.44,222201.0,-47466.0,83792.0,0.67,-17.60,37.71,13.47,138.0,-0.57,1.62,12.39,XY24,NTT,PAINTS
57,RELIANCE,1533.0,-14.04,46.0,H-LC,4.08,215498.0,5132.0,23640.0,0.66,2.44,10.97,13.68,37.0,0.22,1.57,19.37,XY25,NTT,REFINERIES
61,SAMMAANCAP,326.0,-171.59,56.0,M-SC,5.88,82320.0,-19890.0,113281.0,0.58,-19.46,137.61,91.37,208.0,-0.18,0.60,33.63,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-3.65,30.0,H-SC,8.75,114660.0,-10637.0,69564.0,-3.30,-8.49,60.67,47.03,141.0,-0.15,0.84,43.56,MH,ATH,POWER
27,HAPPSTMNDS,1488.71,-20.32,27.0,H-SC,10.52,85920.0,-42245.0,152276.0,-2.90,-32.96,177.23,85.85,132.0,-0.28,0.63,0.00,AR,ATH,IT
77,TTKPRESTIG,770.00,92.96,35.0,M-SC,2.74,83100.0,-17677.0,17767.0,-2.89,-17.54,21.38,0.09,245.0,-0.99,0.61,9.17,OX40N,NTT,DURABLES
54,RAJESHEXPO,518.00,1682.83,40.0,L-SC,2.61,48646.0,-88531.0,88623.0,-2.88,-64.54,182.18,0.07,267.0,-1.00,0.36,20.98,OX40N,NTT,JEWELLERY
39,INDUSINDBK,1800.00,234.73,36.0,L-MC,12.06,41080.0,-36126.0,61521.0,-2.71,-46.79,149.76,32.89,258.0,-0.59,0.30,13.13,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17973.08,-30.15,52.0,H-MC,3.4,202125.0,1305.0,67469.0,-2.26,0.65,33.38,34.25,80.0,0.02,1.48,5.88,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,91.37,48.0,H-SC,10.17,120825.0,-21654.0,41177.0,-0.49,-15.20,34.08,13.70,163.0,-0.53,0.88,42.61,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,92.96,35.0,M-SC,2.74,83100.0,-17677.0,17767.0,-2.89,-17.54,21.38,0.09,245.0,-0.99,0.61,9.17,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,53.0,H-SC,1.44,222201.0,-47466.0,83792.0,0.67,-17.60,37.71,13.47,138.0,-0.57,1.62,12.39,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,36.0,H-SC,1.84,141278.0,-34625.0,76643.0,0.24,-19.68,54.25,23.89,149.0,-0.45,1.03,22.01,OX40N,NTT,CERAMICS
66,SIS,528.0,2077.20,39.0,H-SC,4.23,87873.0,-23159.0,46766.0,-0.43,-20.86,53.22,21.26,156.0,-0.50,0.64,18.73,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,56.0,M-MC,10.25,228854.0,3892.0,163768.0,-2.65,1.73,71.56,74.53,192.0,0.02,1.67,34.54,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.63,47.0,H-MC,3.82,189170.0,24500.0,18822.0,-1.85,14.88,9.95,26.31,89.0,1.30,1.38,24.05,X40N,NTT,AC
84,VOLTAS,1530.00,-3.46,40.0,H-MC,3.03,202515.0,10773.0,26975.0,-1.62,5.62,13.32,19.69,99.0,0.40,1.48,12.80,XY25,NTT,AC
17,CAMS,5211.76,-5.38,45.0,H-SC,1.76,107520.0,5516.0,38406.0,-0.73,5.41,35.72,43.06,122.0,0.14,0.79,23.45,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,53.0,M-LC,6.83,154723.0,13883.0,30604.0,-1.94,9.86,19.78,31.59,66.0,0.45,1.13,38.96,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.91,106954.0,6450.0,22653.0,-1.85,6.42,21.18,28.96,79.0,0.28,0.78,42.56,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,55.0,H-MC,8.23,181992.0,2184.0,110233.0,-1.19,1.21,60.57,62.52,88.0,0.02,1.33,35.72,XR,NTT,BANKS
85,WHIRLPOOL,2270.00,-44.96,31.0,M-SC,1.54,93082.0,1584.0,77165.0,-0.63,1.73,82.90,86.07,223.0,0.02,0.68,35.17,XR,NTT,DURABLES
37,INDIAMART,4850.92,-55.20,25.0,H-SC,10.23,124969.0,1633.0,132130.0,-0.66,1.32,105.73,108.45,119.0,0.01,0.91,23.97,AR,ATH,MISC
86,WIPRO,420.00,-16.52,33.0,M-LC,5.64,146965.0,-3980.0,113442.0,-2.05,-2.64,77.19,72.51,53.0,-0.04,1.07,3.23,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-5.85,28.0,H-LC,12.96,255850.0,-11515.0,76960.0,-1.36,-4.31,30.08,24.48,48.0,-0.15,1.87,48.30,XY24,BTT,FINANCE
74,TCS,4476.75,-31.26,30.0,H-LC,12.07,273963.0,-71993.0,146844.0,-1.45,-20.81,53.60,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
83,VBL,671.64,-20.64,31.0,H-LC,8.19,285856.0,-29986.0,142642.0,-0.84,-9.49,49.90,35.67,5.0,-0.21,2.09,3.05,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.24,31.0,H-LC,7.07,208930.0,-42838.0,96045.0,-1.24,-17.01,45.97,21.13,27.0,-0.45,1.53,12.38,X40,ATH,PAINTS
30,HCLTECH,1943.91,-3.66,35.0,H-LC,9.93,219894.0,-22030.0,85297.0,-1.63,-9.11,38.79,26.15,8.0,-0.26,1.61,3.91,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,40.0,H-LC,1.67,229545.0,-16606.0,95812.0,-2.17,-6.75,41.74,32.18,16.0,-0.17,1.68,26.87,X200,ATH,IT
42,ITC,452.0,-38.77,46.0,H-LC,2.14,198143.0,-1995.0,22886.0,1.27,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.25,41.0,H-LC,2.62,155440.0,-30655.0,78140.0,-1.50,-16.47,50.27,25.51,15.0,-0.39,1.14,14.55,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.04,46.0,H-LC,4.08,215498.0,5132.0,23640.0,0.66,2.44,10.97,13.68,37.0,0.22,1.57,19.37,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.53,42.0,H-LC,4.33,272236.0,6810.0,49983.0,-0.86,2.57,18.36,21.40,11.0,0.14,1.99,10.08,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,30.0,H-LC,12.07,273963.0,-71993.0,146844.0,-1.45,-20.81,53.60,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.37,44.0,H-LC,7.16,224494.0,-9650.0,58279.0,-0.94,-4.12,25.96,20.77,86.0,-0.17,1.64,2.51,X40N,NTT,BREWERIES
83,VBL,671.64,-20.64,31.0,H-LC,8.19,285856.0,-29986.0,142642.0,-0.84,-9.49,49.90,35.67,5.0,-0.21,2.09,3.05,X40N,ATH,FMCG
30,HCLTECH,1943.91,-3.66,35.0,H-LC,9.93,219894.0,-22030.0,85297.0,-1.63,-9.11,38.79,26.15,8.0,-0.26,1.61,3.91,X40,ATH,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.14,198143.0,-1995.0,22886.0,1.27,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.25,41.0,H-LC,2.62,155440.0,-30655.0,78140.0,-1.50,-16.47,50.27,25.51,15.0,-0.39,1.14,14.55,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,46.0,H-LC,2.14,198143.0,-1995.0,22886.0,1.27,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,4.54,204856.0,356.0,41053.0,-0.95,0.17,20.04,20.25,10.0,0.01,1.50,8.92,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.24,31.0,H-LC,7.07,208930.0,-42838.0,96045.0,-1.24,-17.01,45.97,21.13,27.0,-0.45,1.53,12.38,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,46.0,H-LC,4.08,215498.0,5132.0,23640.0,0.66,2.44,10.97,13.68,37.0,0.22,1.57,19.37,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,30.0,H-LC,12.07,273963.0,-71993.0,146844.0,-1.45,-20.81,53.60,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-19.96,42.0,H-LC,8.12,311619.0,-1269.0,172949.0,-1.47,-0.41,55.50,54.87,3.0,-0.01,2.28,6.22,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.14,198143.0,-1995.0,22886.0,1.27,-1.00,11.55,10.44,4.0,-0.09,1.45,4.51,X40,NTT,FMCG
83,VBL,671.64,-20.64,31.0,H-LC,8.19,285856.0,-29986.0,142642.0,-0.84,-9.49,49.90,35.67,5.0,-0.21,2.09,3.05,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,43.0,H-LC,7.97,248040.0,-13579.0,132801.0,-0.64,-5.19,53.54,45.57,7.0,-0.10,1.81,5.54,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,41.46,80333.0,-33216.0,73264.0,-1.25,-29.25,91.20,35.27,268.0,-0.45,0.59,96.62,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,56.0,M-SC,5.88,82320.0,-19890.0,113281.0,0.58,-19.46,137.61,91.37,208.0,-0.18,0.60,33.63,XY25,NTT,FINANCE
49,MASFIN,398.61,-19.60,40.0,H-SC,12.51,91500.0,-6480.0,28081.0,-1.23,-6.61,30.69,22.05,152.0,-0.23,0.67,33.00,XR,ATH,FINANCE
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.91,106954.0,6450.0,22653.0,-1.85,6.42,21.18,28.96,79.0,0.28,0.78,42.56,MH,ATH,METALS
78,UJJIVANSFB,60.00,91.37,48.0,H-SC,10.17,120825.0,-21654.0,41177.0,-0.49,-15.20,34.08,13.70,163.0,-0.53,0.88,42.61,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1174.32,64.0,M-SC,12.70,173250.0,-7893.0,53361.0,-0.39,-4.36,30.80,25.10,235.0,-0.15,1.27,38.84,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-171.59,56.0,M-SC,5.88,82320.0,-19890.0,113281.0,0.58,-19.46,137.61,91.37,208.0,-0.18,0.60,33.63,XY25,NTT,FINANCE
71,TANLA,1963.11,-19.51,55.0,H-SC,16.99,215485.0,-55276.0,385223.0,-2.28,-20.42,178.77,121.86,133.0,-0.14,1.57,68.57,AR,ATH,IT
81,VAIBHAVGBL,521.00,53.17,50.0,H-SC,5.63,132850.0,-49925.0,160988.0,-2.27,-27.32,121.18,60.77,125.0,-0.31,0.97,18.99,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3570.27,48.0,M-SC,7.10,114425.0,-32943.0,84755.0,-1.39,-22.35,74.07,35.16,236.0,-0.39,0.84,16.87,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.30
1,25,43.79
2,50,74.98


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.89
LC    32.50
MC    23.64
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.40
X40      14.77
X40N     11.38
XR       11.14
XY25     10.16
AR        8.31
OX40N     7.90
X200      1.68
MH        1.62
X5K       1.45
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.37
H-LC    25.99
H-MC    20.69
M-SC    13.04
M-LC     5.45
M-MC     2.65
L-SC     1.48
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-6.60,41.79
IT,12.85,-19.62,84.84
FINANCE,10.06,-19.13,71.10
MISC,7.00,-19.83,86.45
BANKS,6.16,-18.31,80.54
PAINTS,5.69,-18.46,36.11
ELECTRICAL,5.47,-9.93,48.50
INSURANCE,3.78,-6.81,46.73
AC,3.57,0.91,15.67


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3219776.0,22
XR,1386185.0,14
AR,1193373.0,9
X40,827559.0,10
X40N,647024.0,9
OX40N,565291.0,10
XY25,447681.0,7
SR,254484.0,2
X5K,119751.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3771753.0,29
M-SC,1460043.0,17
H-MC,1272737.0,15
H-LC,1264523.0,15
M-LC,404375.0,4
M-MC,314950.0,2
L-SC,259850.0,3
L-MC,61521.0,1
L-LC,39401.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1249163.0      6
M-SC       XY24       834697.0      7
H-SC       AR         811832.0      5
           XR         804604.0      7
H-MC       XY24       572824.0      4
H-LC       X40        524021.0      5
H-SC       X40N       318072.0      4
M-MC       XY24       314950.0      2
H-SC       OX40N      264034.0      4
           SR         254484.0      2
H-LC       X40N       241974.0      3
H-MC       X40        227166.0      4
H-LC       AR         210941.0      2
H-MC       XY25       180184.0      2
L-SC       XR         171227.0      2
M-LC       XY24       171182.0      2
M-SC       AR         170600.0      2
           OX40N      139935.0      4
           XR         125158.0      2
M-LC       X5K        119751.0      1
H-LC       XY25       114815.0      3
M-LC       XR         113442.0      1
M-SC       XY25       113281.0      1
H-MC       XR         110233.0      1
H-LC       X200        95812.0      1
L-SC       OX40N       88623.0      1
H-MC       X40N        86978.0      2
H-LC       XY24        76960.0      1
M-SC       X40         76372.0      1
H-MC       OX40N       72699.0      1
H-SC       MH          69564.0      1
L-MC       XR          61521.0      1
L-LC       XY25        39401.0      1
H-MC       MH          22653.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
